In [1]:
import common as ak
from operator_plan import OperatorPlan
from cost_packet import CostPacket
import numpy as np
import numpy.typing as npt

%load_ext autoreload
%autoreload 2

In [2]:
item_dict = ak.get_item_dict(lang="en_US", local=True)
#char_dict = ak.get_char_dict(lang="en_US")
char_dict = ak.get_char_dict(lang="zh_CN", local=True)
module_dict = ak.get_module_dict(lang="zh_CN", local=True)
level_dict = ak.get_level_dict(lang="zh_CN", local=True)

Reading from local file
Reading from local file
Reading from local file
Reading from local file


In [3]:
n_items = len(item_dict.keys())
n_chars = len(char_dict.keys())
item_names, item_names_rev = ak.get_item_info(item_dict)
#char_names, char_names_rev = ak.get_char_info(char_dict)
char_names, char_names_rev = ak.get_char_translations(char_dict)

In [4]:
packet_all, _ = ak.get_all_char_all_costs(char_dict, module_dict, level_dict, n_chars)

In [5]:
plan_nearl = OperatorPlan(
    name = "Kroos",
    level_range = (1, 50),
    elite_range = (0, 1),
    skill_range = (1, 1),
    mastery_range = [
        (0, 0),
        (0, 0),
        (0, 0)
    ],
    module_range = [
        (0, 0),
        (0, 0)
    ]
)

mats_combined = plan_nearl.get_cost(char_names_rev, packet_all)
plan_nearl.pretty_print(item_names, mats_combined)

Kroos:
	E0 -> E1
Cost:
	LMD: 82283
	EXP: 92782


# Testing - Loading Gamepress from JSON

In [6]:
import json
with open("gamepress.json") as f:
    gpdata = json.load(f)["operators"]

In [7]:
n_gp_operators = len(gpdata)
gp_plans = np.empty(n_gp_operators, dtype=OperatorPlan)

In [8]:
for i, op in enumerate(gpdata):
    try:
        ## gamepress id doesnt match in-game id data
        name = char_names_rev[op["operator"]]
    except:
        name = op["operator"]
    #print(name)
    gp_plans[i] = OperatorPlan(
            name = name,
            elite_range = (op["start_promotion"], op["end_promotion"]),
            skill_range = (op["start_skill"], op["end_skill"]),
            mastery_range = np.transpose((op["start_specs"], op["end_specs"]))
        )

# Test - Serialising OperatorPlan objects

In [9]:
obj_str = plan_nearl.to_json()

new_dict = json.loads(obj_str)
new_plan = OperatorPlan(**new_dict)

print(new_plan)

# Test - User interface

In [10]:
preset_e1 = OperatorPlan(elite_range=(0,1))
preset_e1_SL7 = OperatorPlan(elite_range=(0,1), skill_range=(1,7))
preset_e1_L60_SL7 = OperatorPlan(level_range=(1,60), elite_range=(0,1), skill_range=(1,7))
preset_e1_L70_SL7 = OperatorPlan(level_range=(1,70), elite_range=(0,1), skill_range=(1,7))
preset_e2 = OperatorPlan(elite_range=(0,2))
preset_e2_L20_SL7 = OperatorPlan(level_range=(1,20), elite_range=(0,2), skill_range=(1,7))
preset_e2_L50_SL7 = OperatorPlan(level_range=(1,50), elite_range=(0,2), skill_range=(1,7))
preset_e2_SL7 = OperatorPlan(elite_range=(0,2), skill_range=(1,7))
preset_e2_S1M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,3),(0,0),(0,0)])
preset_e2_S2M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,0),(0,3),(0,0)])
preset_e2_S3M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,0),(0,0),(0,3)])
preset_S1M3 = OperatorPlan(mastery_range=[(0,3),(0,0),(0,0)])
preset_S2M3 = OperatorPlan(mastery_range=[(0,0),(0,3),(0,0)])
preset_S3M3 = OperatorPlan(mastery_range=[(0,0),(0,0),(0,3)])
preset_S2M3_S3M3 = OperatorPlan(mastery_range=[(0,0),(0,3),(0,3)])
preset_M6 = OperatorPlan(mastery_range=[(0,3),(0,3),(0,0)])
preset_M9 = OperatorPlan(mastery_range=[(0,3),(0,3),(0,3)])
preset_M1L1 = OperatorPlan(module_range=[(0,1)])
preset_M1L3 = OperatorPlan(module_range=[(0,3)])

In [11]:
suzuran = preset_e2.as_operator("Suzuran")
suzuran_total = suzuran.get_cost(char_names_rev, packet_all)
suzuran.pretty_print(item_names, suzuran_total)

Suzuran:
	E0 -> E2
Cost:
	Polyester: 8
	Oriron: 4
	Grindstone Pentahydrate: 8
	D32 Steel: 4
	Supporter Chip: 5
	Supporter Dualchip: 4
	LMD: 589841
	EXP: 361400


In [12]:
def sum_operator_cost(operators: npt.NDArray, char_names_rev: dict,
                      packet_all: CostPacket) -> npt.NDArray:
    costs = []
    for op in operators:
        op_cost = op.get_cost(char_names_rev, packet_all)
        costs.append(op_cost)
    
    costs = np.concatenate(costs)
    total_cost = ak.collapse_item_list(costs)
    return total_cost

def materials_to_csv(file: str, array: npt.NDArray, item_names: dict) -> None:
    with open(file, "w") as f:
        for item_id, count in array:
            f.write("{},{},{}\n".format(item_id, item_names[item_id], count))
            

        

In [13]:
ops = [
    OperatorPlan("Mostima", level_range=(80, 20), elite_range=(1,2)),
    preset_e2_L50_SL7.as_operator("Thorns"),
    preset_S3M3.as_operator("Thorns"),
    preset_S2M3_S3M3.as_operator("Bagpipe"),
    OperatorPlan("Liskarm", level_range=(70, 20), elite_range=(1,2)),
    OperatorPlan("April", level_range=(70, 20), elite_range=(1,2)),
    OperatorPlan("Utage", level_range=(60, 20), elite_range=(1,2)),
    OperatorPlan("FEater", level_range=(70, 20), elite_range=(1,2)),
    preset_S2M3.as_operator("FEater"),
    OperatorPlan("Shamare", level_range=(70, 20), elite_range=(1,2)),
    OperatorPlan("Elysium", level_range=(70, 20), elite_range=(1,2)),
    preset_M6.as_operator("Elysium"),
    OperatorPlan("Nearl the Radiant Knight", mastery_range=[(0,0),(0,0),(2,3)]),
    preset_e1_L70_SL7.as_operator("Honeyberry"),
    preset_e1_L60_SL7.as_operator("Roberta"),
    OperatorPlan("Aurora", level_range=(70, 20), elite_range=(1,2)),
    OperatorPlan("Archetto", mastery_range=[(2,3),(1,3),(0,3)]),
    OperatorPlan("Mudrock", mastery_range=[(0,0),(1,3),(0,3)]),
    preset_S1M3.as_operator("Saga"),
    preset_S3M3.as_operator("Skadi the Corrupting Heart"),
    preset_S2M3.as_operator("Exusiai"),
    OperatorPlan("Eunectes", mastery_range=[(0,0),(0,0),(2,3)]),
    preset_S2M3.as_operator("Kal'tsit"),
    OperatorPlan("Saria", mastery_range=[(0,0),(1,3),(0,3)]),
    preset_S2M3.as_operator("Skadi"),
    preset_S3M3.as_operator("W"),
    preset_S2M3.as_operator("Ifrit"),
    preset_S2M3.as_operator("Mountain"),
    preset_e2_L50_SL7.as_operator("Ling"),
    preset_S2M3_S3M3.as_operator("Ling"),
    preset_e1_L70_SL7.as_operator("Blacknight"),
    preset_e2_L20_SL7.as_operator("Goldenglow"),
    preset_S2M3_S3M3.as_operator("Goldenglow"),
    OperatorPlan("Dusk", mastery_range=[(1,3),(0,3),(0,3)]),
    OperatorPlan("Eyjafjalla", mastery_range=[(0,0),(1,3),(0,0)]),
    preset_S3M3.as_operator("Aak"),
    OperatorPlan("Weedy", mastery_range=[(1,3)]),
    preset_S2M3.as_operator("Specter"),
    OperatorPlan("Lava the Purgatory", level_range=(70, 20), elite_range=(1,2)),
    preset_e2_L20_SL7.as_operator("Fiammetta"),
    preset_S2M3_S3M3.as_operator("Fiammetta"),
    preset_S3M3.as_operator("Suzuran"),
    preset_e2_L20_SL7.as_operator("Horn"),
    preset_e2_L20_SL7.as_operator("Irene"),
    preset_e2_L20_SL7.as_operator("Specter the Unchained"),
    preset_e2_L20_SL7.as_operator("Lumen"),
    preset_S2M3.as_operator("Phantom"),
    preset_e1_L70_SL7.as_operator("Erato"),
    preset_e1_L70_SL7.as_operator("Kroos the Keen Glint"),
    preset_e1_L70_SL7.as_operator("Rockrock"),
    preset_e2_L20_SL7.as_operator("Kazemaru"),
    preset_e2_L20_SL7.as_operator("Dorothy"),
]
total_cost = sum_operator_cost(ops, char_names_rev, packet_all)
suzuran.pretty_print(item_names, total_cost)

Suzuran:
	E0 -> E2
Cost:
	Orirock: 30
	Orirock Cube: 63
	Orirock Cluster: 85
	Orirock Concentration: 91
	Sugar Substitute: 20
	Sugar: 55
	Sugar Lump: 4
	Ester: 10
	Polyester: 62
	Polyester Pack: 21
	Polyester Lump: 12
	Oriron Shard: 24
	Oriron: 45
	Oriron Cluster: 25
	Oriron Block: 63
	Diketon: 21
	Polyketon: 44
	Aketon: 96
	Keton Colloid: 92
	Damaged Device: 18
	Device: 44
	Integrated Device: 26
	Optimized Device: 40
	Loxic Kohl: 45
	White Horse Kohl: 84
	Manganese Ore: 27
	Manganese Trihydrate: 72
	Grindstone: 29
	Grindstone Pentahydrate: 84
	RMA70-12: 21
	RMA70-24: 79
	Polymerization Preparation: 76
	Bipolar Nanoflake: 82
	D32 Steel: 58
	Crystalline Electronic Unit: 43
	Coagulating Gel: 36
	Polymerized Gel: 87
	Incandescent Alloy: 33
	Incandescent Alloy Block: 86
	Crystalline Component: 4
	Crystalline Circuit: 22
	Semi-Synthetic Solvent: 23
	Refined Solvent: 4
	Compound Cutting Fluid: 21
	Cutting Fluid Solution: 8
	Vanguard Chip: 4
	Vanguard Dualchip: 3
	Guard Chip: 10
	Guard Chip P

In [14]:
materials_to_csv("test.csv", total_cost, item_names)

In [15]:
craft_dict = ak.get_craft_dict(local=True)
#material_ids = ak.get_material_ids(item_names_rev, ak.MATERIAL_NAMES)
material_ids = ak.get_material_ids(item_names_rev, np.concatenate((ak.MATERIAL_NAMES, ak.CHIP_NAMES)))
n_mats = len(material_ids)
craft_matrix, subprod_matrix = ak.get_craft_matrix(craft_dict, material_ids)

Reading from local file


In [16]:
mats_required = np.zeros(n_mats, dtype="int32")
for item_id, count in total_cost:
    try:
        idx = np.where(material_ids == item_id)[0][0]
        mats_required[idx] = count
        #print(item_id)
    except:
        pass
    
#with np.printoptions(threshold=np.inf):
#    print(craft_matrix)
#print(total_cost)
breakdown = np.matmul(craft_matrix.T, mats_required)
print(np.stack([material_ids, breakdown]))

[['4001' '3301' '3302' '3303' '30011' '30012' '30013' '30014' '30061'
  '30062' '30063' '30064' '30031' '30032' '30033' '30034' '30021' '30022'
  '30023' '30024' '30041' '30042' '30043' '30044' '30051' '30052' '30053'
  '30054' '30073' '30074' '30083' '30084' '30093' '30094' '30103' '30104'
  '30115' '30125' '30135' '31013' '31014' '31023' '31024' '31033' '31034'
  '30145' '31043' '31044' '31053' '31054' '3211' '3212' '3213' '3221'
  '3222' '3223' '3231' '3232' '3233' '3241' '3242' '3243' '3251' '3252'
  '3253' '3261' '3262' '3263' '3271' '3272' '3273' '3281' '3282' '3283']
 ['433900.0' '999.0' '3810.0' '0.0' '189.0' '425.0' '602.0' '76.0'
  '132.0' '104.0' '273.0' '82.0' '186.0' '84.0' '159.0' '0.0' '165.0'
  '0.0' '184.0' '0.0' '135.0' '100.0' '301.0' '76.0' '132.0' '384.0'
  '275.0' '76.0' '168.0' '164.0' '240.0' '58.0' '210.0' '58.0' '171.0'
  '58.0' '0.0' '0.0' '0.0' '113.0' '86.0' '195.0' '43.0' '52.0' '43.0'
  '0.0' '4.0' '0.0' '12.0' '0.0' '24.0' '0.0' '0.0' '42.0' '0.0' '0.0'


In [42]:
breakdown_matrix = np.matrix.copy(craft_matrix.T)
tier_1_names = [
    "Damaged Device",
    "Diketon",
    "Oriron Shard",
    "Ester",
    "Sugar Substitute",
    "Orirock",
]
tier_2_names = [
    "Orirock Cube",
    "Device",
    "Polyketon",
    "Oriron",
    "Polyester",
    "Sugar",
]
tier_3_names = [
    "RMA70-12",
    "Manganese Ore",
    "Grindstone",
    "Loxic Kohl",
    "Integrated Device",
    "Aketon",
    "Oriron Cluster",
    "Polyester Pack",
    "Sugar Pack",
    "Orirock Cluster",
    "Coagulating Gel",
    "Incandescent Alloy",
    "Crystalline Component",
    "Semi-Synthetic Solvent",
    "Compound Cutting Fluid",
]

chip_t1 = [
    "Vanguard Chip",
    "Guard Chip",
    "Defender Chip",
    "Sniper Chip",
    "Caster Chip",
    "Medic Chip",
    "Supporter Chip",
    "Specialist Chip",
]

chip_t2 = [
    "Vanguard Chip Pack",
    "Guard Chip Pack",
    "Defender Chip Pack",
    "Sniper Chip Pack",
    "Caster Chip Pack",
    "Medic Chip Pack",
    "Supporter Chip Pack",
    "Specialist Chip Pack",
]

dualchips = [
    "Vanguard Dualchip",
    "Guard Dualchip",
    "Defender Dualchip",
    "Sniper Dualchip",
    "Caster Dualchip",
    "Medic Dualchip",
    "Supporter Dualchip",
    "Specialist Dualchip",
]


tier_1_ids = ak.get_material_ids(item_names_rev, tier_1_names)
tier_2_ids = ak.get_material_ids(item_names_rev, tier_2_names)
tier_3_ids = ak.get_material_ids(item_names_rev, tier_3_names)
ids_combined = np.concatenate((tier_1_ids, tier_2_ids))
ids_combined, ids_indices, _ = np.intersect1d(material_ids, ids_combined, return_indices=True)
_, tier_3_indices, _ = np.intersect1d(material_ids, tier_3_ids, return_indices=True)

chip_ids = ak.get_material_ids(item_names_rev, ak.CHIP_NAMES)
chip_t1_ids = ak.get_material_ids(item_names_rev, chip_t1)
chip_t2_ids = ak.get_material_ids(item_names_rev, chip_t2)
dualchip_ids = ak.get_material_ids(item_names_rev, dualchips)
_, chip_t1_indices, _ = np.intersect1d(material_ids, chip_t1_ids, return_indices=True)
_, chip_t2_indices, _ = np.intersect1d(material_ids, chip_t2_ids, return_indices=True)
_, dualchip_indices, _ = np.intersect1d(material_ids, dualchip_ids, return_indices=True)


rocc2_idx = np.where(material_ids == item_names_rev["Orirock Cube"])[0][0]
rocc3_idx = np.where(material_ids == item_names_rev["Orirock Cluster"])[0][0]
lmd_idx = np.where(material_ids == ak.LMD_ID)[0][0]

## remove deconstructing T3->T2 and T2->T1
rocc_before = breakdown_matrix[rocc2_idx][rocc3_idx]
for i in ids_indices:
    for j in range(n_mats):
        breakdown_matrix[i][j] = 0
        #breakdown_matrix[j][i] = 0
    breakdown_matrix[i][i] = 1

## remove lmd cost
#for j in range(n_mats):
#    breakdown_matrix[lmd_idx][j] = 0

## make T3 mats and LMD persist
breakdown_matrix[lmd_idx][lmd_idx] = 1
for i in tier_3_indices:
    if i == rocc3_idx:
        breakdown_matrix[rocc2_idx][rocc3_idx] = rocc_before
        continue
    breakdown_matrix[i][i] = 1

## remove swapping T1 chip types
for i in chip_t1_indices:
    for j in range(n_mats):
        breakdown_matrix[i][j] = 0
        #breakdown_matrix[j][i] = 0
    breakdown_matrix[i][i] = 1

## remove swapping T2 chip types
for i in chip_t2_indices:
    for j in range(n_mats):
        breakdown_matrix[i][j] = 0
        #breakdown_matrix[j][i] = 0
    breakdown_matrix[i][i] = 1
    
## TODO: move to craft matrix
## add dualchip crafting
for i, k in enumerate(dualchip_indices):
    breakdown_matrix[chip_t2_indices[i]][k] = 2

In [43]:
def csv_to_materials(file: str, mats: npt.NDArray, item_names: dict) -> npt.NDArray:
    raw = np.genfromtxt(file, delimiter=",", dtype=[("item_id", "U32"), ("_","U32"), ("count", "int32")])
    ret = np.zeros(len(mats), dtype=[("item_id", "U32"), ("count", "int32")])
    
    ret["item_id"] = mats
    for i in range(len(mats)):
        try:
            mat_idx = np.where(raw["item_id"] == mats[i])[0][0]
            ret["count"][i] = raw["count"][mat_idx]
        except Exception as e:
            print("Error: {}\n{}".format(e, item_names[mats[i]]))
        
    return ret


#material_ids_with_chips = ak.get_material_ids(item_names_rev, np.concatenate((ak.MATERIAL_NAMES, ak.CHIP_NAMES)))
current_inv = csv_to_materials("mats.csv", material_ids, item_names)
for a, b in np.stack([material_ids, current_inv["count"]]).T:
    print("{}: {}".format(item_names[a], b))

LMD: 0
Skill Summary - 1: 351
Skill Summary - 2: 110
Skill Summary - 3: 53
Orirock: 352
Orirock Cube: 247
Orirock Cluster: 209
Orirock Concentration: 2
Damaged Device: 124
Device: 221
Integrated Device: 14
Optimized Device: 1
Ester: 221
Polyester: 182
Polyester Pack: 57
Polyester Lump: 1
Sugar Substitute: 230
Sugar: 295
Sugar Pack: 8
Sugar Lump: 0
Oriron Shard: 194
Oriron: 180
Oriron Cluster: 440
Oriron Block: 5
Diketon: 218
Polyketon: 186
Aketon: 342
Keton Colloid: 14
Loxic Kohl: 297
White Horse Kohl: 46
Manganese Ore: 234
Manganese Trihydrate: 3
Grindstone: 249
Grindstone Pentahydrate: 10
RMA70-12: 159
RMA70-24: 25
Polymerization Preparation: 0
Bipolar Nanoflake: 0
D32 Steel: 5
Coagulating Gel: 256
Polymerized Gel: 0
Incandescent Alloy: 0
Incandescent Alloy Block: 17
Crystalline Component: 4
Crystalline Circuit: 4
Crystalline Electronic Unit: 0
Semi-Synthetic Solvent: 15
Refined Solvent: 0
Compound Cutting Fluid: 21
Cutting Fluid Solution: 1
Vanguard Chip: 14
Vanguard Chip Pack: 6
Va

In [49]:
#breakdown = mats_required
#mats_remaining = mats_required - current_inv["count"]
mats_remaining = mats_required
breakdown = np.matmul(breakdown_matrix, np.matmul(breakdown_matrix, mats_remaining))
breakdown = np.matmul(breakdown_matrix, breakdown)
for k, v in np.stack([material_ids, breakdown]).T:
    #if np.parseint(v) > 0:
        print("{}: {}".format(item_names[k], v))
#print(np.stack([material_ids, breakdown]))

#print(breakdown[rocc_idx])

LMD: 10604866.0
Skill Summary - 1: 0.0
Skill Summary - 2: 0.0
Skill Summary - 3: 0.0
Orirock: 30.0
Orirock Cube: 6418.0
Orirock Cluster: 0.0
Orirock Concentration: 0.0
Damaged Device: 18.0
Device: 44.0
Integrated Device: 558.0
Optimized Device: 0.0
Ester: 10.0
Polyester: 62.0
Polyester Pack: 314.0
Polyester Lump: 0.0
Sugar Substitute: 20.0
Sugar: 55.0
Sugar Pack: 424.0
Sugar Lump: 0.0
Oriron Shard: 24.0
Oriron: 45.0
Oriron Cluster: 622.0
Oriron Block: 0.0
Diketon: 21.0
Polyketon: 44.0
Aketon: 581.0
Keton Colloid: 0.0
Loxic Kohl: 435.0
White Horse Kohl: 0.0
Manganese Ore: 459.0
Manganese Trihydrate: 0.0
Grindstone: 422.0
Grindstone Pentahydrate: 0.0
RMA70-12: 414.0
RMA70-24: 0.0
Polymerization Preparation: 0.0
Bipolar Nanoflake: 0.0
D32 Steel: 0.0
Coagulating Gel: 278.0
Polymerized Gel: 0.0
Incandescent Alloy: 400.0
Incandescent Alloy Block: 0.0
Crystalline Component: 142.0
Crystalline Circuit: 0.0
Crystalline Electronic Unit: 0.0
Semi-Synthetic Solvent: 27.0
Refined Solvent: 0.0
Compou

In [34]:

chip_idx = np.where(material_ids == chip_ids[2])[0][0]
with np.printoptions(threshold=np.inf):
    stack = np.stack([material_ids, breakdown_matrix[:][chip_idx]]).T
    #print(stack)
    print(item_names[chip_ids[2]])
    for a, b in stack:
        print("{}: {}".format(item_names[a], b))
    #print(craft_matrix[lmd_idx][:])

Vanguard Dualchip
LMD: 0.0
Skill Summary - 1: 0.0
Skill Summary - 2: 0.0
Skill Summary - 3: 0.0
Orirock: 0.0
Orirock Cube: 0.0
Orirock Cluster: 0.0
Orirock Concentration: 0.0
Damaged Device: 0.0
Device: 0.0
Integrated Device: 0.0
Optimized Device: 0.0
Ester: 0.0
Polyester: 0.0
Polyester Pack: 0.0
Polyester Lump: 0.0
Sugar Substitute: 0.0
Sugar: 0.0
Sugar Pack: 0.0
Sugar Lump: 0.0
Oriron Shard: 0.0
Oriron: 0.0
Oriron Cluster: 0.0
Oriron Block: 0.0
Diketon: 0.0
Polyketon: 0.0
Aketon: 0.0
Keton Colloid: 0.0
Loxic Kohl: 0.0
White Horse Kohl: 0.0
Manganese Ore: 0.0
Manganese Trihydrate: 0.0
Grindstone: 0.0
Grindstone Pentahydrate: 0.0
RMA70-12: 0.0
RMA70-24: 0.0
Polymerization Preparation: 0.0
Bipolar Nanoflake: 0.0
D32 Steel: 0.0
Coagulating Gel: 0.0
Polymerized Gel: 0.0
Incandescent Alloy: 0.0
Incandescent Alloy Block: 0.0
Crystalline Component: 0.0
Crystalline Circuit: 0.0
Crystalline Electronic Unit: 0.0
Semi-Synthetic Solvent: 0.0
Refined Solvent: 0.0
Compound Cutting Fluid: 0.0
Cuttin

In [21]:
print(a)

3283
